# Building an Advanced AI Agent for Thai Cuisine with Agno Framework

In today's article, I'll walk you through creating an advanced AI agent specialized in Thai cuisine using the Agno framework. This agent combines document understanding and web search capabilities to deliver comprehensive cooking advice and cultural information about Thai food.

## Understanding the Agno Framework

The [Agno framework](https://github.com/agno-ai/agno) provides a modular approach to building AI agents with specialized knowledge and capabilities. Let's explore how we can leverage this framework to create a Thai cuisine expert.

## Setting Up the Environment

First, we need to import the necessary components and set up our environment:

In [1]:
from agno.agent import Agent  # Import the main Agent class
from agno.models.openai import OpenAIChat  # Import the OpenAI chat model
from agno.embedder.openai import OpenAIEmbedder  # Import OpenAI embedding model
from agno.tools.duckduckgo import DuckDuckGoTools  # Import web search capability
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase  # Import PDF knowledge base
from agno.vectordb.lancedb import LanceDb, SearchType  # Import vector database
import os  # For environment variables
from dotenv import load_dotenv  # For loading environment variables from .env file
load_dotenv(override=True)  # Load environment variables from .env file

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")  # Set OpenAI API key

This setup imports all the necessary components and loads our API key from a .env file.

## Creating the Thai Cuisine Agent

Now, let's create our specialized agent:


In [2]:
agent = Agent(  # Create an agent instance
    model=OpenAIChat(id="gpt-4o-mini"),  # Use GPT-4o-mini model
    description="You are a Thai cuisine expert!",  # Set agent's role
    instructions=[  # Define agent's behavior
        "Search your knowledge base for Thai recipes.",  # Primary source is knowledge base
        "If the question is better suited for the web, search the web to fill in gaps.",  # Use web as backup
        "Prefer the information in your knowledge base over the web results."  # Prioritize knowledge base
    ],
    knowledge=PDFUrlKnowledgeBase(  # Configure knowledge source
        urls=["https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf"],  # URL to Thai recipes PDF
        vector_db=LanceDb(  # Vector database configuration
            uri="tmp/lancedb",  # Local database storage path
            table_name="recipes",  # Table name for storing embeddings
            search_type=SearchType.hybrid,  # Use hybrid search for better results
            embedder=OpenAIEmbedder(id="text-embedding-3-small"),  # Embedding model
        ),
    ),
    tools=[DuckDuckGoTools()],  # Enable web search with DuckDuckGo
    show_tool_calls=True,  # Display tool usage in output
    markdown=True  # Format responses as markdown
)

Let's break down what this agent configuration does:

1. **Base Model**: We're using `gpt-4o-mini`, OpenAI's efficient smaller version of their GPT-4o model
2. **Agent Role**: We've defined the agent as a Thai cuisine expert
3. **Knowledge Base**: The agent has access to a PDF document containing Thai recipes
4. **Vector Database**: LanceDB stores and retrieves document embeddings using a hybrid search approach
5. **Web Search**: DuckDuckGo integration allows the agent to search the web for additional information
6. **Output Formatting**: Markdown support for better readability

## Loading the Knowledge Base and Testing

Next, we need to load our knowledge base and test the agent with some queries:

In [3]:
# Comment out after the knowledge base is loaded
if agent.knowledge is not None:  # Check if knowledge base exists
    agent.knowledge.load()  # Load and index the PDF content

# First query with markdown enabled
agent.print_response("How do I make chicken and galangal in coconut milk soup", stream=True)  # Query about Tom Kha Gai

# For the second query, try-except block to handle potential markdown issues
try:
    agent.print_response("What is the history of Thai curry?", stream=True)  # Try query with markdown
except Exception as e:  # Handle potential markdown parsing errors
    print("\nEncountered an error with markdown formatting. Retrying without markdown...\n")  # Error message
    # Create a temporary agent with markdown disabled for this query
    temp_agent = Agent(  # Create alternative agent
        model=OpenAIChat(id="gpt-4o-mini"),  # Use same model
        description="You are a Thai cuisine expert!",  # Same role
        instructions=[  # Same instructions
            "Search your knowledge base for Thai recipes.",
            "If the question is better suited for the web, search the web to fill in gaps.",
            "Prefer the information in your knowledge base over the web results."
        ],
        knowledge=agent.knowledge,  # Reuse the already loaded knowledge base
        tools=[DuckDuckGoTools()],  # Same tools
        show_tool_calls=True,  # Same settings
        markdown=False  # Disable markdown formatting
    )
    temp_agent.print_response("What is the history of Thai curry?", stream=True)  # Retry query without markdown

INFO Loading knowledge base

INFO Reading: https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf

INFO No documents to insert

INFO Added 0 documents to knowledge base

Output()

Output()


This code:

1. Loads and indexes the Thai recipes PDF into the vector database
2. Tests the agent with a recipe question (how to make Tom Kha Gai)
3. Handles potential markdown formatting issues with a try-except block
4. Creates a fallback agent with markdown disabled if needed

## Key Technical Features

This implementation demonstrates several advanced techniques:

1. **Document Understanding**: The agent processes and indexes a PDF document, allowing it to reference specific Thai recipes.

2. **Hybrid Search**: The `SearchType.hybrid` setting combines keyword and semantic search for more accurate document retrieval.

3. **Knowledge Prioritization**: The agent is instructed to prefer its knowledge base over web search results, ensuring recipe authenticity.

4. **Error Handling**: The code includes fallback mechanisms for potential markdown formatting issues.

5. **Streaming Responses**: The `stream=True` parameter provides a more interactive user experience with progressive responses.

## Use Cases for This Agent

This Thai cuisine expert agent could be deployed in various scenarios:

- **Cooking Applications**: Integrate it into cooking apps for authentic Thai recipe guidance
- **Cultural Education**: Use it in educational platforms about Thai culinary traditions
- **Restaurant Assistance**: Help restaurants develop authentic Thai menus
- **Dietary Adaptation**: Assist users in adapting Thai recipes for dietary restrictions

## Extending the Agent

You could enhance this agent by:

1. Adding more knowledge sources (additional PDFs about Thai regional cuisines)
2. Implementing image generation capabilities to visualize dishes
3. Adding a measurement conversion tool for international users
4. Incorporating user feedback to improve recipe recommendations

## Conclusion

The Agno framework makes it straightforward to create specialized AI agents with domain expertise. Our Thai cuisine expert demonstrates how to combine document knowledge with web search capabilities to create a helpful culinary assistant.

This approach can be adapted for many different domains - from legal research to technical support - by changing the knowledge sources and agent instructions while keeping the same underlying architecture.

What specialized AI agent would you like to build with this framework? Let me know in the comments!